# Simulation analytics - Orchestrating 4C simulations with QUEENS

Until now, all the simulation models we used in QUEENS were Python based. Now for a lot of us, this is not the case. From [fenics](https://fenicsproject.org/) to [abaqus](https://www.3ds.com/de/products/simulia/abaqus), we all have our own preferred solvers. One main advantage of QUEENS is its independence of solver type. This means, to use your QUEENS model with your favorite solver, the only thing you have to do is write an interface.

Since QUEENS and the multiphysics solver 4C share some developers, the QUEENS community already provides a ready-to-go interface. So let's start there.

# The 4C model

In this example, we'll have a look at a nonlinear solid mechanics problem. The momentum equation is given by

$$\rho \ddot{\boldsymbol{u}} = \nabla \cdot \boldsymbol{\sigma} + \boldsymbol{b} \text{ in } \Omega$$

where $u$ are the displacements, $\boldsymbol{\sigma}$ the Cauchy stress and $b$ volumetric body forces. For the examples, $\boldsymbol{b}=\boldsymbol{0}$, and we prescribe Dirichlet boundary conditions
$$ \boldsymbol{u} = \boldsymbol{0} \text{ on } \Gamma_w = \{z=-2.5\}$$
$$ \boldsymbol{u} = [0, 0, 0.55] \text{ on } \Gamma_e =\{z=2.5\}$$

and a Neumann condition:
$$ \boldsymbol{\sigma} \cdot \boldsymbol{n} = \boldsymbol{0} \text{ on } \Gamma \setminus \{\Gamma_w \cup \Gamma_e\}$$

We assume a static problem, so $\ddot{\boldsymbol{u}} = 0$, and use a Saint Venant–Kirchhoff material model
$$ \boldsymbol{S} = \lambda \text{tr}(\boldsymbol{E})\boldsymbol{I} + 2 \mu \boldsymbol{E}$$
where $\boldsymbol{S}$ is the second Piola-Kirchhoff stress tensor and $\boldsymbol{E}$ the Green-Lagrange strain tensor with the Lamé parameters
$$\lambda = \frac{E\nu}{(1+\nu)(1-2\nu)}$$
$$\mu = \frac{E}{2(1+\nu)}$$
where $E$ is the Young's modulus and $\nu$ the Poisson ratio.

The problem is discretized in space using 40 linear finite elements.

> Note: This tutorial is based on the 4C tutorial [Preconditioners for Iterative Solvers](https://4c-multiphysics.github.io/4C/documentation/tutorials/tutorial_preconditioning.html)

> Note: If you are using the docker container, plotting with pyvista might not work. In that case, set `pyvista_available` to `False`, and please ask your peers in the group to show and discuss their results. You can also refer to the pdf solution file for figures.

In [ ]:
# Set this to False if pyvista is causing the kernel to crash 
pyvista_available = True

In [ ]:
# Let's look at the mesh!
if pyvista_available:
    import pyvista as pv

    fe_mesh = pv.read("beam_coarse.exo")

    pl = pv.Plotter("Finite Element mesh")
    pl.add_mesh(fe_mesh, show_edges=True, opacity=0.8)
    pl.add_mesh(pv.Plane([0, 0, -2.50001], [0, 0, 1]), color="blue")  # Gamma_w
    pl.add_mesh(pv.Plane([0, 0, 2.50001], [0, 0, 1]), color="red")  # Gamma_e
    pl.add_axes(line_width=5)
    pl.camera_position = [
        (1.1321899035097993, -6.851600196807601, 2.7649096132703574),
        (0.0, 0.0, 0.2749999999999999),
        (-0.97637930372977, -0.08995062285804697, 0.19644933366041056),
    ]
    pl.show()

The red plane is $\Gamma_e$, the blue one is $\Gamma_w$.

# Simulation Analytics

Now, we want to have a look at the Cauchy stresses $\boldsymbol{\sigma}$ for various Young's modulus $E$ and Poisson ratio $\nu$. For the purpose of demonstration, let's assume a uniform distribution for each material parameter.

Of course we use QUEENS, so let's get started!

In [ ]:
from queens.distributions import Uniform
from queens.parameters import Parameters

# Let's define the distributions
young = Uniform(10, 10000)
poisson_ratio = Uniform(0, 0.5)

# Let's define the parameters
parameters = Parameters(young=young, poisson_ratio=poisson_ratio)

Now that we have the parameters ready, we need a `Driver`. In QUEENS terms, drivers call the underlying simulation models, provide the simulation input and handle logging and error management. They are the interfaces to arbitrary simulation software and essentially do everything that is needed to start a **single** simulation.

As mentioned before, for 4C these are already provided by the QUEENS community.

> Note: Comment in the lines of code that fits with your configuration (docker / virtual machine)

In [ ]:
from queens_interfaces.fourc.driver import Fourc
import pathlib


# # Set the paths for the docker configuration
# home = pathlib.Path("/home/user")
# fourc_executable = "/home/user/4C/build/4C"
# input_template = "template_docker.4C.yaml"

# # Set the paths for the virtual machine configuration
# home = pathlib.Path("/home/participant")
# fourc_executable = "/home/participant/4C/build/release/4C"
# input_template = "template.4C.yaml"

fourc_driver = Fourc(
    parameters=parameters,  # parameters to run
    input_templates=input_template,  # input template
    executable=fourc_executable,  # 4C executable
)

4C simulations are controlled via yaml input files. They need to be created for every simulation. Therefore, we provide a [jinja2](https://jinja.palletsprojects.com/en/stable/) input **template** to the driver. Jinja2 is a fast, expressive, and extensible templating engine. It starts from a template, for example

```yaml
MATERIALS:
  - MAT: 1
    MAT_Struct_StVenantKirchhoff:
      YOUNG: {{ young }}
      NUE: {{ poisson_ratio }}
      DENS: 1
```

and inserts the desired values (here: `young = 1000` and `poisson_ratio = 0.3`) into the respective placeholders (here: `{{ young }}` and `{{ poisson_ratio }}`). This results in:
```yaml
MATERIALS:
  - MAT: 1
    MAT_Struct_StVenantKirchhoff:
      YOUNG: 1000
      NUE: 0.3
      DENS: 1
```

For 4C, this allows us to create inputs for simulation runs without the need for a 4C API! Let's run this example!

In [ ]:
import pathlib
import numpy as np

driver_output = fourc_driver.run(
    sample=np.array([1000, 0.3]),
    job_id="fourc_young_1000_poisson_ratio_03",
    num_procs=1,
    experiment_dir=pathlib.Path(".."),
    experiment_name="fourc_run",
)

Some information:
- `sample`: is the inputs for which we want to evaluate the model. The order of the data is given by the order of which we define the parameters.
- `job_id`: is the identifier for the 4C run and also creates a folder with the same name where the data is stored.
- `num_procs`: 4C is called using MPI. This is the number of processes that 4C will be called with.
- `experiment_dir`: is the directory where we find all necessary data for 4C (input template or any other required files).
- `experiment_name`: is an identifier of a QUEENS run. This will come in handy when we evaluate 4C multiple times.

Let's look at the output folder structure of the QUEENS run:

```bash
fourc_young_1000_poisson_ratio_03/
├── input_file.yaml
├── jobscript.sh
├── metadata.yaml
└── output
    ├── output.control
    ├── output.log
    ├── output.mesh.structure.s0
    ├── output-structure.pvd
    └── output-vtk-files
        ├── structure-00000-0.vtu
        ├── structure-00000.pvtu
        ├── structure-00001-0.vtu
        └── structure-00001.pvtu
```

What are those files?
- `input_file.yaml`: is the 4C input file with $E=1000$ and $\nu=0.3$
- `jobscript.sh`: contains the command with which 4C was called
- `metadata.yaml`: contains metadata of the 4C (timings, inputs, outputs, ...)
- `output`-folder: all the files in here are output files of 4C! The sole exception is `output.log` where the logging data was redirected to this file by QUEENS

Go have a look at the `yaml`, `sh` and log files.

Now let us look at the Cauchy stresses $\boldsymbol{\sigma}$

In [ ]:
# This is just needed for some nice plots

from utils import plot_results
if pyvista_available:
    plotter = pv.Plotter()
    plot_results(
        "fourc_young_1000_poisson_ratio_03/output/output-vtk-files/structure-00001.pvtu",
        plotter,
    )
    plotter.show()

We plotted the deformed mesh. The original mesh is displayed as the blue 'wireframe'. Let us look into the driver output in Python.

In [ ]:
print("Driver output:", driver_output)

As we can see it's a tuple of two `None`. The first one is supposed to be the output of the model, and the second one the model gradient.

> Note: The gradient value is only available for certain models, this particular 4C model can't do so.

But why is it `None`?

Well, we told the driver how to run a 4C simulation, but never told it which information we want to extract. For this we need a data processor, i.e., an object that is able to extract data from the simulation results.

In [ ]:
from queens.data_processors import PvdFile


# Create custom data processor from the available PVD file reader
class CustomDataProcessor(PvdFile):
    def __init__(
        self,
        field_name,
        failure_value,
        file_name_identifier=None,
        files_to_be_deleted_regex_lst=None,
    ):
        super().__init__(
            field_name,
            file_name_identifier,
            file_options_dict={},
            files_to_be_deleted_regex_lst=files_to_be_deleted_regex_lst,
            time_steps=[-1],
            block=0,
            point_data=False,  # We use cell data
        )
        self.failure_value = failure_value

    def get_data_from_file(self, base_dir_file):

        # Get the data
        data = super().get_data_from_file(base_dir_file)

        # Got some data
        if data is not None:
            data = data.flatten()
            # Simulation started but only the first step was exported
            if np.abs(data).sum() < 1e-8:
                return self.failure_value

            # Return result
            return data

        # Simulation could not even start
        else:
            return self.failure_value


custom_data_processor = CustomDataProcessor(
    field_name="element_cauchy_stresses_xyz",  # Name of the field in the pvtu file
    file_name_identifier="output-structure.pvd",  # Name of the file in the output folder
    failure_value=np.nan * np.ones(40 * 6),  # NaN for all elements
)

We defined our data processor, so let's continue.

In [ ]:
# Create a new driver
fourc_driver = Fourc(
    parameters=parameters,
    input_templates=input_template,
    executable=fourc_executable,
    data_processor=custom_data_processor,  # tell the driver to extract the data
)

# Run the driver again
driver_output, _ = fourc_driver.run(
    sample=np.array([1000, 0.3]),
    job_id="fourc_young_1000_poisson_ratio_03",
    num_procs=1,
    experiment_dir=pathlib.Path(".."),
    experiment_name="fourc_run",
)

print("Driver output", driver_output)
print(
    "Number of entries",
    len(driver_output),
    "i.e., 40 elements x 6 Cauchy stress values",
)

What we did:

- We defined a driver using QUEENS to run 4C simulation as a Python function call
- We provided a custom way of extracting simulations results
- Although we selected a single process, setting  `num_procs>1` allows us to run a simulation in parallel without having to change anything in the interface!

**The complexity of working with a sophisticated simulation code (without an API) is hidden from the user!**

**Instead, it is reduced to `fourc_driver.run(<arguments>)`**

Now in the day-to-day of computational mechanics research, we often want to study convergence of a model depending on certain parameters. In this example, we'll have a look at parameter combinations of $E \in [10,10000]$ and  $\nu \in [0,0.5]$ which lead to a failing simulation. Let's evaluate the 4C model on a grid of $4 \times 21$ points.

In [ ]:
# Make it less verbose
import logging

logging.getLogger("distributed").setLevel(logging.WARN)

from queens.schedulers import Local
from queens.iterators import Grid
from queens.models import Simulation
from queens.global_settings import GlobalSettings
from queens.main import run_iterator
from queens.utils.io import load_result

n_grid_points_young = 4
n_grid_points_poisson_ratio = 21

grid_output = None
if __name__ == "__main__":
    pathlib.Path("grid").mkdir(exist_ok=True)
    with GlobalSettings(
        experiment_name=f"grid_{n_grid_points_young}x{n_grid_points_poisson_ratio}",
        output_dir="grid",
    ) as gs:
        scheduler = Local(
            gs.experiment_name,
            num_jobs=4,  # 4 simulations in parallel
            num_procs=1,  # each simulation uses 1 process
        )
        model = Simulation(scheduler, fourc_driver)

        iterator = Grid(
            model=model,
            parameters=parameters,
            global_settings=gs,
            result_description={"write_results": True},
            grid_design={
                "young": {
                    "num_grid_points": n_grid_points_young,
                    "axis_type": "log10",
                    "data_type": "FLOAT",
                },
                "poisson_ratio": {
                    "num_grid_points": n_grid_points_poisson_ratio,
                    "axis_type": "lin",
                    "data_type": "FLOAT",
                },
            },
        )

        run_iterator(iterator, gs)

        grid_output = load_result(gs.result_file("pickle"))

In [ ]:
# Let's plot the data
grid_points = grid_output["input_data"]
cauchy_stresses = grid_output["raw_output_data"]["result"]

import matplotlib.pyplot as plt

converged_samples = []
failed_samples = []
failed = []
for input_sample, cauchy_output in zip(grid_points, cauchy_stresses, strict=True):
    if np.isnan(cauchy_output).any():
        failed_samples.append(input_sample)
        failed.append(1)
    else:
        converged_samples.append(input_sample)
        failed.append(0)

converged_samples = np.array(converged_samples)
failed_samples = np.array(failed_samples)
failed = np.array(failed)

fig, ax = plt.subplots(figsize=(12, 8))
ax.contourf(
    grid_points[:, 0].reshape(21, 4),
    grid_points[:, 1].reshape(21, 4),
    failed.reshape(21, 4),
    levels=[0, 0.000001],
)

for i, gp in enumerate(grid_points):
    plt.text(gp[0] * 1.04, gp[1], i)

ax.scatter(converged_samples[:, 0], converged_samples[:, 1], c="b", label="Converged")
ax.scatter(failed_samples[:, 0], failed_samples[:, 1], c="r", label="Failed")

ax.set_xlim(10 * 0.9, 10000 * 1.2)
ax.set_yticks(np.linspace(0, 0.5, 11))
ax.set_ylim(-0.01, 0.515)
ax.set_xlabel("Young's module $E$")
ax.set_xscale("log")
ax.set_ylabel("Poisson ratio $\\nu$")
fig.suptitle("Grid study 4C")
ax.legend()
fig.tight_layout()
plt.show()

Each point represents one simulation, i.e., one 4C run with different parameters. The number next to it is the job id, so the unique identifier for this simulation for QUEENS. The color of the points:
- blue: The simulation ran through and returned a value.
- red: The simulation failed and NaN was returned.

A colored background indicates the region of the parameter space leads to converging results.

Previously, we called 4C using the driver directly. However, when using QUEENS iterators, this won't work anymore. Instead, we need a `Model`. The difference is that a driver handle a single simulation, whereas a model accepts a list of relevant input samples / configuration.

When working with drivers, the `Simulation` model is used. Besides a driver, this requires a `Scheduler`. As the name suggests, schedulers schedule evaluations of the driver. The scheduler also handles the evaluation of multiple drivers calls at the same time. Keep in mind that each driver call can itself be parallelized with MPI, such that QUEENS natively supports nested parallelism!

Since we are doing our computations locally, we use a `Local` scheduler. However, there is also a `Cluster` scheduler which is able to submit jobs on high performance clusters using `SLURM`, `PBS` etc.

The scheduler also creates the required folder structure for an experiment. The default value is set to `~/queens-experiments/<experiment_name>`. For the grid iterator, it looks like this:
```bash
grid_4x21/
├── 0
├── 1
...
├── 80
├── 81
├── 82
├── 83
├── 9
└── template.4C.yaml
```

Each number is a job with its output:
```bash
10
├── input_file.yaml
├── jobscript.sh
├── metadata.yaml
└── output
    ├── output.control
    ├── output.log
    ├── output.mesh.structure.s0
    ├── output-structure.pvd
    └── output-vtk-files
        ├── structure-00000-0.vtu
        ├── structure-00000.pvtu
        ├── structure-00001-0.vtu
        └── structure-00001.pvtu
```

Once more, have a look at the files. In particular look at the log files of the failed simulations.

In [ ]:
# Let's look at some simulations outputs!
if pyvista_available:
    job_ids = [0, 41, 50, 75]

    plotter = pv.Plotter(shape=(2, 2))

    for i in range(2):
        for j in range(2):
            plotter.subplot(i, j)
            job_id = job_ids[i * 2 + j]
            file_path = (
                home
                / f"queens-experiments/grid_4x21/{job_id}/output/output-vtk-files/structure-00001.pvtu"
            )
            plotter.add_text(f"Job: {job_id}")
            plot_results(
                file_path,
                plotter,
                f"E={grid_points[job_id][0]}, nu={np.round(grid_points[job_id][1],decimals=4)}: Cauchy stress zz\n",
            )
    plotter.show()

In summary, we employed QUEENS to do some simulation analytics of a 4C model. We identified failed simulations, looked at the outputs and extracted data.

# Calibration

Now, let's imagine we have some experimental measurements of the Cauchy stresses at the element centers $\boldsymbol{\sigma}_\text{ex}$ of a real life cantilever. We can now use this, to calibrate our parameters $E$ and $\nu$ from experiment.

> Note: We assume that the 4C model is able to predict the real-word accurately, i.e, we have the right boundary conditions, constitutive law etc. 

To do so, we'll use deterministic optimization to minimize the difference between simulated and measured stresses:
$$(E, \nu) \in \underset{E, \nu}{\text{argmin}} \ \frac{1}{n_\text{ele}} \sum_{e=1}^{n_\text{ele}} (\boldsymbol{\sigma}_e(E, \nu)-\boldsymbol{\sigma}_{\text{ex},e}):(\boldsymbol{\sigma}_e(E, \nu)-\boldsymbol{\sigma}_{\text{ex},e})$$

where $\boldsymbol{\sigma}_e(E,\nu)$ are the elementwise stresses obtained from the 4C model evaluated at $E, \nu$, where the supscript $e$ is the element id. 

Before we start the optimization, we need to change our parameter space. As we previously identified, the simulation model does not converge for $\nu > 0.45$. So in order to make meaningful predictions, let's change the parameter space.

In [ ]:
# Reduced parameter space for the poisson ratio
poisson_ratio = Uniform(0, 0.45)

parameters = Parameters(young=young, poisson_ratio=poisson_ratio)

In [ ]:
# Let's implement the loss function

from queens.models._model import Model


class LossFunction(Model):
    def __init__(self, experimental_cauchy_stresses, forward_model):
        self.experimental_cauchy_stresses = experimental_cauchy_stresses
        self.forward_model = forward_model
        super().__init__()

    def _evaluate(self, samples):
        element_cauchy_stresses = self.forward_model.evaluate(samples)["result"]

        difference = element_cauchy_stresses - np.tile(
            self.experimental_cauchy_stresses, (len(samples), 1)
        )
        squared_norm = (difference**2).sum(axis=1) + (difference[:, 3:] ** 2).sum(
            axis=1
        )  # the output is only a symmetrical tensor, so we have to add the offdiagonal terms once more

        squared_norm /= 40  # 40 elements

        return {"result": squared_norm.reshape(-1)}

    def grad(self, samples, upstream_gradient):
        pass


from experimental_data import experimental_elementwise_cauchy_stress

Ok, now let's set up the optimization using the [L-BFGS-B](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html) algorithm.

In [ ]:
from scipy.optimize import Bounds
from queens.iterators import Optimization

optimum = None

if __name__ == "__main__":
    pathlib.Path("optimization").mkdir(exist_ok=True)
    with GlobalSettings(
        experiment_name="optimization_young_poisson_ratio", output_dir="optimization"
    ) as gs:
        scheduler = Local(
            gs.experiment_name,
            num_jobs=4,  # 4 simulations in parallel
            num_procs=1,  # each simulation uses 1 process
        )
        custom_data_processor = CustomDataProcessor(
            field_name="element_cauchy_stresses_xyz",  # Name of the field in the pvtu file
            file_name_identifier="output-structure.pvd",  # Name of the file in the output folder
            failure_value=np.nan * np.ones(40 * 6),  # NaN for all elements
            files_to_be_deleted_regex_lst=[
                "output.control",
                "*.mesh.s0",
                "output-vtk-files/*",
            ],  # Delete these files after we extracted the data.
        )
        fourc_driver = Fourc(
            parameters=parameters,
            input_templates=input_template,
            executable=fourc_executable,
            data_processor=custom_data_processor,  # tell the driver to extract the data
        )
        model = Simulation(scheduler, fourc_driver)
        loss_function = LossFunction(experimental_elementwise_cauchy_stress, model)
        optimization = Optimization(
            loss_function,
            parameters,
            gs,
            initial_guess=[200, 0.4],
            result_description={"write_results": True},
            objective_and_jacobian=True,
            bounds=Bounds([10, 0], [10000, 0.45]),
            algorithm="L-BFGS-B",
            max_feval=1000,
        )

        run_iterator(optimization, gs)

        optimum = optimization.solution.x

In [ ]:
# Let's run the optimum:

fourc_driver.data_processor = None

fourc_driver.run(
    optimum,
    experiment_dir=pathlib.Path(".."),
    job_id="calibrated_value",
    num_procs=1,
    experiment_name=None,
)


def tensor_norm(tensor_data):
    data = np.hstack(
        (tensor_data.tolist(), tensor_data[:, 3:].tolist())
    )  # duplicate offdiagonal terms
    return np.sqrt((data**2).sum(axis=1)).reshape(-1, 1)


def plot_tensor_norm(output_mesh, field_name, data, plotter, color_bar_title):
    output_mesh[field_name] = data

    plotter.add_mesh(
        output_mesh,
        scalars=field_name,
        scalar_bar_args={
            "title": color_bar_title,
            "title_font_size": 15,
            "label_font_size": 15,
        },
    )
    plotter.add_axes(line_width=5)
    plotter.camera_position = [
        (1.1321899035097993, -6.851600196807601, 2.7649096132703574),
        (0.0, 0.0, 0.2749999999999999),
        (-0.97637930372977, -0.08995062285804697, 0.19644933366041056),
    ]

if pyvista_available:
    output_mesh = pv.read("calibrated_value/output/output-vtk-files/structure-00001.pvtu")
    output_calibrated_value = output_mesh["element_cauchy_stresses_xyz"]
    difference = output_calibrated_value - experimental_elementwise_cauchy_stress.reshape(
        -1, 6
    )
    relative_error = tensor_norm(difference) / tensor_norm(
        experimental_elementwise_cauchy_stress.reshape(-1, 6)
    )
    plotter = pv.Plotter()
    plot_tensor_norm(
        output_mesh,
        "relative",
        relative_error,
        plotter,
        "Relative error of the Cauchy stress norm difference between calibrated value and experimental data\n",
    )
    plotter.add_text(
        f"E={np.round(optimum[0], decimals=2)}, nu={np.round(optimum[1], decimals=2)}",
        "upper_right",
    )
    plotter.show()

We successfully conducted a calibration of the Young's modulus and Poisson ratio $\nu$! The largest relative error
$$\frac{\sqrt{(\boldsymbol{\sigma}_e(E, \nu)-\boldsymbol{\sigma}_{\text{ex},e}):(\boldsymbol{\sigma}_e(E, \nu)-\boldsymbol{\sigma}_{\text{ex},e})}}{\sqrt{\boldsymbol{\sigma}_{\text{ex},e}:\boldsymbol{\sigma}_{\text{ex},e}}}$$
between calibrated model output and experimental data is $< 1.7\cdot10^{-8}$.

This means we have a good state estimation, i.e., the model output matches the provided data! In fact, the data was not experimental at all, but was generated with $E=123$ and $\nu=0.4$, which the optimizer identified quickly.

# Why QUEENS?

- Hide complexity in handling complex simulation codes:
  - `Driver`: make 4C callable via a Python function call
  - `Scheduler`: schedule and handle simulations in parallel
  - `Model`: hide the scheduler and driver logic for the QUEENS user
- Nested parallelism
  - Multiple processes per 4C run
  - Multiple parallel 4C runs at the same time
- Modularity:
  - Switched from grid evaluation to optimization simply by changing the iterator
  - 4C simulation model did not change!

## Let's play around

Let your creativity flow and try out stuff.

### Inspirations
- Change the parameters ranges
- Look at the 4C input 
- Look at the driver outputs
- Why did the simulations fail? (hint: two different reasons)
- Plot the displacement field